# extracting line, paragraph, and page level ocr data

https://en.wikipedia.org/wiki/HOCR

In [7]:
import bs4
soup = bs4.BeautifulSoup(Path("/Users/tonyfast/Downloads/fashions-handee_hocr.html").read_text(), features="html.parser")
soup = bs4.BeautifulSoup(Path("/Users/tonyfast/Downloads/accessiblity-guidebook-outdoor-trails_hocr.html").read_text(), features="html.parser")

load ocr data as an array

In [8]:
elements = Series(soup.body.find_all(), name="element")

tidy the data

In [9]:
df = elements.apply(
    compose_left(operator.attrgetter("attrs"), Series)
).join(elements)

classes = (
    df["class"]
    .dropna()
    .explode()
    .str.removeprefix("ocr_")
    .str.removeprefix("ocrx_")
)

assert len(classes.index.unique()) == len(classes)

df = df.drop(columns="class").join(classes.rename("klass"))
df = df.assign(
    page=df.klass.eq("page").cumsum(),
    par=df.klass.eq("par").cumsum(),
    word=df.klass.eq("word").cumsum(),
    line=df.klass.eq("line").cumsum()
)
page_line = pl = df.groupby("page").line.apply(lambda s: s - s.min())
df = df.assign(page_line = Series(pl.values, pl.index.get_level_values(1), name="page_line"))
df = df.join(df[(df.klass == "word")].element.apply(bs4.Tag.get_text).rename("letters"))
df.sample(5).T

,9921,47712,3055,62150,41960
id,line_000021_000061,line_000085_000031,word_000008_000051,word_000113_000055,word_000076_000182
title,bbox 262 2214 1178 2249; baseline -0.001 -7; x...,bbox 226 2440 1192 2478; baseline 0 -9; x_size...,bbox 288 946 425 975; x_wconf 96; x_fsize 38,bbox 790 1065 825 1080; x_wconf 96; x_fsize 21,bbox 1526 759 1671 778; x_wconf 96; x_fsize 38
lang,NaN,NaN,NaN,NaN,NaN
xml:lang,NaN,NaN,NaN,NaN,NaN
element,"[\n, [e], \n, [Residences], \n, [e], \n, [Fire...","[\n, [millimeters)], \n, [above,], \n, [the], ...",[Outdoor],[firm],[someone]
klass,line,line,word,word,word
page,22,86,9,114,77
par,297,1615,79,2193,1436
word,8390,39299,2646,51012,34514
line,906,4348,256,5715,3832


combine the words on each line

In [10]:
    lines = df.groupby("page par page_line".split()).letters.agg(
        compose(lambda x: x or None, " ".join, Series.dropna)
    ).dropna()
    lines.sample(5).to_frame()

letters
page par  page_line                                                   
41   652  1          Applying the Forest Service Outdoor Recreation...
45   753  35         so they are firm and stable. Other soils can b...
          37         pearance. Many surface materials that appear n...
7    73   26         WANN CANS: jaca5scnek cet cetteey angle, eet g...
118  2335 17         trail, representing its intended design and ma...

aggregate the page content

In [11]:
    (
        pages := lines
            .groupby("page par".split())
            .agg("\n".join)
            .groupby("page")
            .agg("\n\n".join)
    )
    for page, text in pages.items(): display(Markdown(F"### Page {page}")), print(text)

### Page 1

United States
Department of
Agriculture

Forest Service

Technology &
Development
Program

2300—Recreation
August 2012
1223-2806P—MTDC

U.S, Department of Transportation

Federal Highway Administration

USDA

Accessibility Guidebook
for Outdoor Recreation
and Trails


### Page 2

Notice

This publication was produced in cooperation with the U.S. Department of
Transportation, Federal Highway Administration's Recreational Trails Program in
the interest of information exchange. The U.S. Government assumes no liability
for the use of the information and does not endorse products or manufacturers.
Trademarks or manufacturers’ names and Web sites appear only because they are
considered essential to the objective of this publication.

The contents of this publication reflect the views of the authors, who are responsible
for the facts and accuracy of the information presented herein. The contents do not
necessarily reflect the official policy of the U.S. Department of Transportation. This
report does not constitute a standard, specification, or regulation.

ae

U.S. Department of Transportation

Federal Highway Administration

You can order a copy of this publication using the order form on the
FHWA’s Recreational Trails Program Web site at:
http://www.fhwa.dot.gov/env

### Page 3

...

Appendix: FSTAG Implementation Process Flowchart

122


### Page 131

About the Authors

Janet Zeller is the national accessibility program man-
ager for the Forest Service and is based in Washington, DC.
She began her Forest Service career in 1991 and has worked
in the field of accessibility for more than two decades. She is
the team leader for the Forest Service accessibility program
and for the development and implementation of the program
and accessibility policies. She also teaches accessibility and
universal design for programs and facilities at a wide range of
training sessions nationally. She holds a bachelor’s degree in
education from the University of New Hampshire and a mas-
ter’s degree from the University of Rhode Island.

Ruth Doyle is the recreation, engineering, archeology,
lands, and minerals staff officer at the Cibola National For-
est. She was the Forest Service’s representative on the U.S.
Access Board’s Outdoor Developed Areas Regulatory Nego-
tiation Committee. This committee developed the preliminary
accessibility guidelines for p

### Page 133

Accessibility Guidebook for Outdoor Recreation and Trails
